In [1]:
import numpy as np
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import tensorflow as tf
import numpy as np

n_labels = 6

GDFS_path = "/content/drive/My Drive/Colab Notebooks/fake_lstm/"

def sample(arr, labels, seq):
	sample_sentences = []
	sample_labels = []
	for i in range(len(arr)):
		tokens = len(arr[i])
		if tokens < seq:
			inst = np.array(arr[i],dtype=np.float32)
			zeros = np.ones((seq-tokens, 60))
			inst = np.concatenate([inst, zeros], axis=0)
			sample_sentences.append(inst)
			sample_labels.append(labels[i])
	sample_sentences = np.array(sample_sentences, dtype=np.float32)
	sample_labels = np.array(sample_labels, dtype=np.int32)
	onehot_labels = np.eye(n_labels)[sample_labels]
	print(sample_sentences.shape)
	print(onehot_labels.shape)    
	return np.nan_to_num(sample_sentences,copy=False), onehot_labels


def load_data(type):
	train_sentences = np.load(GDFS_path+"train_multi_glove_m.npy", allow_pickle=True)
	train_labels = np.load(GDFS_path+"train_"+type+"_labels_m.npy", allow_pickle=True)
	val_sentences = np.load(GDFS_path+"val_"+type+"_glove_m.npy", allow_pickle=True)
	val_labels = np.load(GDFS_path+"val_"+type+"_labels_m.npy", allow_pickle=True)
	test_sentences = np.load(GDFS_path+"test_"+type+"_glove_m.npy", allow_pickle=True)
	test_labels = np.load(GDFS_path+"test_"+type+"_labels_m.npy", allow_pickle=True)
	return train_sentences, train_labels, test_sentences, test_labels, val_sentences, val_labels


train_sentences, train_labels, test_sentences, test_labels, val_sentences, val_labels = load_data("binary")




seq_length = 200

epochs = 10

batch_size = 32

batches = train_labels.shape[0]//batch_size

test_batches = test_labels.shape[0]//batch_size

val_batches = val_labels.shape[0]//batch_size


train_vects, train_labels = sample(train_sentences, train_labels, seq_length)
test_vects, test_labels = sample(test_sentences, test_labels, seq_length)
val_vects, val_labels = sample(val_sentences, val_labels, seq_length)




x = tf.placeholder(tf.float32, [None, seq_length, 60], name="x")
y = tf.placeholder(tf.float32, [None, n_labels], name="y")

inpt = tf.transpose(x, [1,0,2])


def lstm_cell(lstm_size, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    return lstm

cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(128, 0.75) for _ in range(4)])

state = cell.zero_state(batch_size, tf.float32)

outputs, final_state = tf.nn.dynamic_rnn(cell, inpt, initial_state=state, time_major=True)

output_final = tf.squeeze(tf.slice(outputs, [seq_length-1,0,0] , [1,-1,-1]))

fc1 = tf.layers.dense(output_final, units=256, activation=tf.nn.leaky_relu)

logit = tf.squeeze(tf.layers.dense(fc1, units=n_labels, activation=None))

loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logit, labels=y))

op = tf.train.AdamOptimizer(0.0001).minimize(loss)

pred = tf.argmax(tf.nn.sigmoid(logit), axis=1)

actual = tf.argmax(y, axis=1)

acc = tf.reduce_mean(tf.to_float(tf.equal(pred, actual)))



saver = tf.train.Saver()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  val_accs = 0
  for ep in range(epochs):
    accs = []
    lss = []

    for batch in range(batches//2):
      random_indexes = np.random.choice(train_labels.shape[0], batch_size)
      x_batch = train_vects[random_indexes]
      y_batch = train_labels[random_indexes]
      fd = {x: x_batch, y: y_batch}
      _, ls, ac = sess.run([op, loss, acc], fd)
      accs.append(ac)
      lss.append(ls)
    print("epoch : ", ep, ", loss : ", np.mean(lss), ", acc : ", np.mean(accs))
    accs = []
    lss = []
    for batch in range(val_batches//2):
      random_indexes = np.random.choice(val_labels.shape[0], batch_size)
      x_batch = val_vects[random_indexes]
      y_batch = val_labels[random_indexes]
      fd = {x: x_batch, y: y_batch}
      ls, ac = sess.run([loss, acc], fd)
      accs.append(ac)
      lss.append(ls)
    print("val set | loss : ", np.mean(lss), ", acc : ", np.mean(accs))
    if np.mean(accs) >= val_accs:
      saver.save(sess, "./model_ckpt")
      val_accs = np.mean(accs)
  accs = []
  saver.restore(sess, "./model_ckpt")
  print("saved modes val acc : ", val_accs)
  for batch in range(test_batches):
    random_indexes = np.random.choice(test_labels.shape[0], batch_size)
    x_batch = test_vects[random_indexes]
    y_batch = test_labels[random_indexes]
    fd = {x: x_batch, y: y_batch}
    ls, ac = sess.run([loss, acc], fd)
    accs.append(ac)
  print("test set | acc : ", np.mean(accs))

(10143, 200, 60)
(10143, 6)
(1248, 200, 60)
(1248, 6)
(1274, 200, 60)
(1274, 6)


W0805 07:33:09.457991 140504847247232 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0805 07:33:09.459727 140504847247232 deprecation.py:323] From <ipython-input-2-bd4c69e63927>:69: BasicLSTMCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
W0805 07:33:09.477757 140504847247232 deprecation.py:323] From <ipython-input-2-bd4c69e63927>:73: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions

epoch :  0 , loss :  0.29002124 , acc :  0.54472655
val set | loss :  0.23564076 , acc :  0.53125
epoch :  1 , loss :  0.22971621 , acc :  0.5638672
val set | loss :  0.23207767 , acc :  0.5625
epoch :  2 , loss :  0.23077154 , acc :  0.5466797
val set | loss :  0.23058537 , acc :  0.5359375
epoch :  3 , loss :  0.22988883 , acc :  0.55507815
val set | loss :  0.23338473 , acc :  0.496875
epoch :  4 , loss :  0.2289855 , acc :  0.5625
val set | loss :  0.23230596 , acc :  0.525
epoch :  5 , loss :  0.23018412 , acc :  0.5457031
val set | loss :  0.23378846 , acc :  0.484375
epoch :  6 , loss :  0.23047373 , acc :  0.5408203
val set | loss :  0.23083112 , acc :  0.5234375
epoch :  7 , loss :  0.22865196 , acc :  0.5666016
val set | loss :  0.23173526 , acc :  0.5109375
epoch :  8 , loss :  0.22898726 , acc :  0.56171876
val set | loss :  0.23175402 , acc :  0.509375
epoch :  9 , loss :  0.22786012 , acc :  0.5740234


W0805 07:44:24.145426 140504847247232 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


val set | loss :  0.23445508 , acc :  0.5421875
saved modes val acc :  0.5625
test set | acc :  0.5608974
